In [35]:
import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")

users = pd.read_json("users.json")

In [36]:
print(orders)

      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                     restaurant_name  
0                  New Foods Chinese  
1     Ruchi Curry House Multicuisine  
2              Spice Kitchen Punjabi  
3             Darbar Kitche

In [37]:
print(users)

      user_id       name       city membership
0           1     User_1    Chennai    Regular
1           2     User_2       Pune       Gold
2           3     User_3  Bangalore       Gold
3           4     User_4  Bangalore    Regular
4           5     User_5       Pune       Gold
...       ...        ...        ...        ...
2995     2996  User_2996  Hyderabad       Gold
2996     2997  User_2997  Hyderabad    Regular
2997     2998  User_2998  Bangalore    Regular
2998     2999  User_2999       Pune    Regular
2999     3000  User_3000  Hyderabad       Gold

[3000 rows x 4 columns]


In [38]:
# Create SQLite database locally
conn = sqlite3.connect("restaurant.db")
cursor = conn.cursor()

# Execute SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

# Read restaurants table
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


OperationalError: table restaurants already exists

In [39]:
print(restaurants)

     restaurant_id restaurant_name  cuisine  rating
0                1    Restaurant_1  Chinese     4.8
1                2    Restaurant_2   Indian     4.1
2                3    Restaurant_3  Mexican     4.3
3                4    Restaurant_4  Chinese     4.1
4                5    Restaurant_5  Chinese     4.8
..             ...             ...      ...     ...
495            496  Restaurant_496   Indian     3.1
496            497  Restaurant_497  Mexican     4.4
497            498  Restaurant_498  Chinese     3.9
498            499  Restaurant_499  Mexican     4.9
499            500  Restaurant_500  Chinese     3.2

[500 rows x 4 columns]


In [40]:
# STEP 4: LEFT JOINS (MANDATORY)

# Orders + Users
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)

# Orders + Users + Restaurants
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

# STEP 5: Save Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("Final dataset created successfully!")

Final dataset created successfully!


In [41]:

df = pd.read_csv("final_food_delivery_dataset.csv")

gold_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(gold_revenue)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [42]:
avg_order_value = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(avg_order_value)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [43]:


df = pd.read_csv("final_food_delivery_dataset.csv")

# Total spend per user
user_total = (
    df.groupby("user_id")["total_amount"]
    .sum()
)

# Count users with total spend > 1000
count_users = (user_total > 1000).sum()

print(count_users)


2544


In [44]:

df = pd.read_csv("final_food_delivery_dataset.csv")

# Create rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 - 3.5", "3.6 - 4.0", "4.1 - 4.5", "4.6 - 5.0"]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels
)


In [45]:
revenue_by_rating = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(revenue_by_rating)


rating_range
4.6 - 5.0    2197030.75
4.1 - 4.5    1960326.26
3.0 - 3.5    1881754.57
3.6 - 4.0    1717494.41
Name: total_amount, dtype: float64


C:\Users\somes\AppData\Local\Temp\ipykernel_5676\4273448026.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


In [46]:
df = pd.read_csv("final_food_delivery_dataset.csv")

gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(gold_city_avg)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [47]:
df = pd.read_csv("final_food_delivery_dataset.csv")

restaurant_count = (
    df.groupby("cuisine")["restaurant_id"]
    .nunique()
    .sort_values()
)

print(restaurant_count)


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64


In [48]:
revenue_by_cuisine = (
    df.groupby("cuisine")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(revenue_by_cuisine)

cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64


In [49]:
total_orders = len(df)


In [50]:
gold_orders = len(df[df["membership"] == "Gold"])

In [51]:
percentage = (gold_orders / total_orders) * 100
print(round(percentage))

50


In [52]:
print(df.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')


In [53]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

subset = (
    df[df["restaurant_name_x"].isin(options)]
    .groupby("restaurant_name_x")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

filtered = subset[subset["total_orders"] < 20]
result = filtered.sort_values("avg_order_value", ascending=False)

print(result)

                     total_orders  avg_order_value
restaurant_name_x                                 
Ruchi Foods Chinese            19       686.603158


In [54]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(combo_revenue)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [55]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

In [56]:
df["quarter"] = df["order_date"].dt.to_period("Q")


In [57]:
quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(quarter_revenue)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64


In [58]:

#Convert order_date to datetime
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)



#How many total orders were placed by Gold members?
def total_orders_by_gold_members():
    count = df[df["membership"] == "Gold"].shape[0]
    print("Total orders placed by Gold members:")
    print(f"→ {count} orders\n")
    return count


#Total revenue from orders placed in Hyderabad
def total_revenue_hyderabad():
    revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
    revenue_rounded = round(revenue)
    print("Total revenue from Hyderabad (rounded):")
    print(f"→ {revenue_rounded}\n")
    return revenue_rounded


#How many distinct users placed at least one order?
def distinct_users_with_orders():
    count = df["user_id"].nunique()
    print("Distinct users who placed at least one order:")
    print(f"→ {count} users\n")
    return count



#    Average order value for Gold members
def average_order_value_gold_members():
    avg_value = df[df["membership"] == "Gold"]["total_amount"].mean()
    avg_value_rounded = round(avg_value, 2)
    print("Average order value for Gold members:")
    print(f"→ {avg_value_rounded}\n")
    return avg_value_rounded



# MAIN EXECUTION
if __name__ == "__main__":
    print("\n ANSWERS FROM FINAL DATASET\n")

    total_orders_by_gold_members()
    total_revenue_hyderabad()
    distinct_users_with_orders()
    average_order_value_gold_members()



 ANSWERS FROM FINAL DATASET

Total orders placed by Gold members:
→ 4987 orders

Total revenue from Hyderabad (rounded):
→ 1889367

Distinct users who placed at least one order:
→ 2883 users

Average order value for Gold members:
→ 797.15



In [59]:

#     How many orders were placed for restaurants
#     with rating >= 4.5 ?
def orders_with_high_rating():
    count = df[df["rating"] >= 4.5].shape[0]
    print("Orders placed for restaurants with rating ≥ 4.5:")
    print(f"{count} orders\n")
    return count


#     How many orders were placed in the top revenue city
#     among Gold members only?
def orders_in_top_gold_city():
    # Step 1: Find top revenue city among Gold members
    gold_city_revenue = (
        df[df["membership"] == "Gold"]
        .groupby("city")["total_amount"]
        .sum()
        .sort_values(ascending=False)
    )

    top_city = gold_city_revenue.index[0]

    # Step 2: Count orders in that city (Gold members only)
    order_count = df[
        (df["membership"] == "Gold") &
        (df["city"] == top_city)
    ].shape[0]

    print("Orders placed in top revenue city among Gold members:")
    print(f"City: {top_city}")
    print(f"Orders: {order_count}\n")

    return order_count


# MAIN EXECUTION
if __name__ == "__main__":
    print("\nADDITIONAL NUMERICAL ANSWERS\n")
    orders_with_high_rating()
    orders_in_top_gold_city()


ADDITIONAL NUMERICAL ANSWERS

Orders placed for restaurants with rating ≥ 4.5:
3374 orders

Orders placed in top revenue city among Gold members:
City: Chennai
Orders: 1337

